### ***Librerias***

In [1]:
# Librerias para trabajar con los datos.
import pandas as pd
import numpy as  np
import yfinance as yf

# Librerias para graficar
import plotly.express as px

### ***Carga de la data***

In [2]:
# Importamos la tabla del URL
url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
df_list = pd.read_html(url)
empresas = df_list[0]

# Carga de la data com la API de Yahoo finance
data = yf.download(empresas['Symbol'].tolist(), start='2000-01-01', end='2021-12-31')

# Ordenamos los Datos por indices (Fechas)
data.sort_index(inplace = True)

[*********************100%***********************]  503 of 503 completed

3 Failed downloads:
- BF.B: No data found for this date range, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- CEG: Data doesn't exist for startDate = 946695600, endDate = 1640919600


### ***- Mejor día para invertir teniendo en cuenta el retorno de los movimiento gap***

In [3]:
# Calculamos los retornos gap
retornos_gaps = np.log(data['Open']/data['Close'].shift(1)).fillna(0)

# Asignamos el dia de la semana correspondiente con cada una de las fechas
retornos_gaps.index = pd.to_datetime(retornos_gaps.index)
retornos_gaps['Dia'] = retornos_gaps.index.day_name()

# Agrupamos por dia y calculamos el promedio
retornos_gaps_promedio = retornos_gaps.groupby('Dia').mean().sum(axis=1)

# Generamos un nuevo DataFrame con nuestros resultados
gaps = pd.DataFrame()
gaps['Dias'] = retornos_gaps_promedio.index
gaps['Retornos Gap'] = retornos_gaps_promedio.values
gaps = gaps.sort_values(['Dias'])
gaps

,Dias,Retornos Gap
0,Friday,0.099351
1,Monday,0.034065
2,Thursday,0.025167
3,Tuesday,0.257419
4,Wednesday,0.051491


Con los resultados obtenidos se puede suponer que los Martes son el mejor día para invertir, ya que es el día con el retorno de los movimientos gap mas alto.

### ***- Promedio anual de los retornos gap***

In [4]:
# Generamos un nuevo DF para graficar
gap_anual = pd.DataFrame()

# Guardamos el promedio diario de los retornos GAP
gap_anual['Promedio'] = retornos_gaps.mean(axis=1)

# Reseteamos el indice y fijamos las fechas en un formato anual
gap_anual.reset_index(inplace=True)
gap_anual['Date'] = gap_anual['Date'].dt.strftime('%Y')

# Agrupamos anualmente y obtenemos el promedio
gap_anual = gap_anual.groupby(['Date']).mean()
gap_anual.reset_index(inplace=True)

# Graficamos
fig = px.area(gap_anual, x='Date', y='Promedio', width=600, height=400)
fig.update_traces(line_color='#17BECF', line_width=3)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

C:\Users\usuario\AppData\Local\Temp\ipykernel_19132\1756718537.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  gap_anual['Promedio'] = retornos_gaps.mean(axis=1)


### ***- Mejor día para invertir teniendo en cuenta el retorno de los movimientos intradiarios***

In [5]:
# Calculamos los retornos intradiarios
retornos_intra = np.log(data['Close']/data['Open']).fillna(0)

# Utilizamos la fecha que tenemos como indice para generar los nombres de los distintos dias de la semana
retornos_intra.index = pd.to_datetime(retornos_intra.index)
retornos_intra['Dia'] = retornos_intra.index.day_name()

# Agrupamos por dias y calculamos el valor promedio
retornos_intra_promedio = retornos_intra.groupby(by=['Dia']).mean().sum(axis=1)

# Generamos un nuevo DataFrame con nuestros resultados
intradiarios = pd.DataFrame()
intradiarios['Dias'] = retornos_intra_promedio.index
intradiarios['Retornos Intradiarios'] = retornos_intra_promedio.values
intradiarios

,Dias,Retornos Intradiarios
0,Friday,0.111415
1,Monday,-0.137661
2,Thursday,0.232219
3,Tuesday,0.082819
4,Wednesday,0.107670


Con los resultados obtenidos se puede suponer que segun los retornos intradiarios el mejor dia para invertir son los Jueves, ya que este es el dia con el indice mas alto.

### ***- Promedio anual de los retornos intradiarios***

In [6]:
# Generamos un nuevo DF para graficar
intra_anual = pd.DataFrame()

# Guardamos el promedio diario de los retornos intradiarios.
intra_anual['Promedio'] = retornos_intra.mean(axis=1)

# Reseteamos el indice y fijamos las fechas en un formato anual
intra_anual.reset_index(inplace=True)
intra_anual['Date'] = intra_anual['Date'].dt.strftime('%Y')

# Agrupamos anualmente y obtenemos el promedio
intra_anual = intra_anual.groupby(['Date']).mean()
intra_anual.reset_index(inplace=True)

# Graficamos
fig = px.area(intra_anual, x='Date', y='Promedio', width=600, height=400)
fig.update_traces(line_color='#17BECF', line_width=3)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

C:\Users\usuario\AppData\Local\Temp\ipykernel_19132\2144924105.py:5: FutureWarning:

Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.



### ***- Mejores industrias que pertenecen al SP500 en las cuales se puede invertir***

In [7]:
# Calculamos la variacion
variacion = data['Adj Close'].pct_change()
variacion_promedio = variacion.mean()

# Creamos un nuevo DF para cargar la variacion en nuestro DF de empresas  
variacion_df = pd.DataFrame()
variacion_df['Variacion Promedio'] = variacion_promedio.values
variacion_df['Symbol'] = variacion_promedio.index

# Utilizamos el metodo merge() para agregar la variacion promedio
empresas = empresas.merge(variacion_df, on='Symbol', copy=False)

# Agrupamos por Sector y obtenemos los 5 promedios mas altos
top_industrias = empresas.groupby(by=['GICS Sector']).sum().sort_values(by=['Variacion Promedio'],ascending=False).head(5)

# Eliminamos la columna CIK que no es relevante para este caso
top_industrias.drop('CIK', axis = 1, inplace=True)

# Reseteamos indice.
top_industrias.reset_index(inplace=True)

# Mostramos en pantalla los 5 mejores sectores para invertir segun la variacion
top_industrias

,GICS Sector,Variacion Promedio
0,Information Technology,0.072342
1,Industrials,0.058783
2,Health Care,0.056137
3,Consumer Discretionary,0.053565
4,Financials,0.044960


In [8]:
# Graficamos
fig = px.bar(top_industrias, x='GICS Sector', y='Variacion Promedio', width=600, height=400)
fig.update_traces(marker_color='#17BECF')
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.show()

### ***- Momentos de alta volatilidad que afectaron al SP500***

In [9]:
# Calculamos la volatilidad
volatilidad = variacion.rolling(250).std()*100*(250)**0.5

# Calculamos su promedio y lo ordenamos
volatilidad = volatilidad.mean(axis = 1)
volatilidad = volatilidad.sort_index()

# Graficamos
x = volatilidad.index
y = volatilidad.values
fig = px.area(volatilidad, x=x, y=y, width=600, height=400)
fig.update_traces(line_color='#17BECF', line_width=3)
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20),
)
fig.update_xaxes(rangeslider_visible=True)
fig.show()

Segun la grafica, se puede observar unos periodos de alta volatilidad en los años 2001, 2009 y 2021.

### ***- Cuales son las 9 mejores empresas para invertir***

In [10]:
# Creamos un DataFrame nuevo
df_empresas = pd.DataFrame()

# Obtengo la variacion diaria
variacion = data['Adj Close'].pct_change()

# Obtengo el promedio de variacion de cada empresa
variacion_promedio = variacion.mean()

# Me guardo las 9 con mayor promedio
top_empresas = variacion_promedio.sort_values(ascending=False).head(9)

# Obtengo sus indices para utilizarlos como mascara
top_empresas = top_empresas.index

# Utilizo la mascara
top_empresas = variacion[top_empresas]

# Obtengo el promedio
top_empresas = top_empresas.mean()

# Guardo los datos en el nuevo DataFrame para luego graficar
df_empresas['Empresas'] = top_empresas.index
df_empresas['Variacion Historica Promedio'] = top_empresas.values

# Ordeno de mayor a menor
df_empresas.sort_values(by='Variacion Historica Promedio', ascending=False, inplace=True)

# Graficamos
fig = px.bar(df_empresas, x='Empresas', y='Variacion Historica Promedio', width=600, height=400)
fig.update_traces(marker_color='#17BECF')
fig.update_layout(
    margin=dict(l=20, r=20, t=20, b=20)
)
fig.show()

Con los resultados obtenidos podemos suponer que las 9 mejores empresas para invertir son las siguientes:

- Moderna, Inc. (MRNA)

- Carrier Global Corporation (CARR)

- Enphase Energy, Inc. (ENPH)

- Tesla, Inc. (TSLA)

- Caesars Entertainment, Inc. (CZR)

- SolarEdge Technologies, Inc. (SEDG)

- Paycom Software, Inc. (PAYC)

- Netflix, Inc. (NFLX)

- Etsy, Inc. (ETSY)